# 2 версия

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from glob import glob

TRAIN_IMG_DIR = "/content/drive/MyDrive/data/train/image"
TRAIN_MASK_DIR = "/content/drive/MyDrive/data/train/label"

VAL_IMG_DIR = "/content/drive/MyDrive/data/val/image"
VAL_MASK_DIR = "/content/drive/MyDrive/data/val/label"

print("Train images:", len(glob(TRAIN_IMG_DIR + "/*.npy")))
print("Train masks:", len(glob(TRAIN_MASK_DIR + "/*.npy")))
print("Val images:", len(glob(VAL_IMG_DIR + "/*.npy")))
print("Val masks:", len(glob(VAL_MASK_DIR + "/*.npy")))

# Пример вывода нескольких файлов для проверки
print("Train images sample:", glob(TRAIN_IMG_DIR + "/*.npy")[:5])
print("Train masks sample:", glob(TRAIN_MASK_DIR + "/*.npy")[:5])

Train images: 2975
Train masks: 2975
Val images: 445
Val masks: 500
Train images sample: ['/content/drive/MyDrive/data/train/image/2788.npy', '/content/drive/MyDrive/data/train/image/2784.npy', '/content/drive/MyDrive/data/train/image/2782.npy', '/content/drive/MyDrive/data/train/image/2778.npy', '/content/drive/MyDrive/data/train/image/2774.npy']
Train masks sample: ['/content/drive/MyDrive/data/train/label/2783.npy', '/content/drive/MyDrive/data/train/label/2790.npy', '/content/drive/MyDrive/data/train/label/2789.npy', '/content/drive/MyDrive/data/train/label/2777.npy', '/content/drive/MyDrive/data/train/label/279.npy']


In [ ]:
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from glob import glob
import os
from tqdm import tqdm
import segmentation_models_pytorch as smp

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 4
NUM_CLASSES = 19
LR = 1e-3
EPOCHS = 25

Шаг 4. Определение аугментаций и датасета

In [ ]:
train_transform = A.Compose([
    A.Resize(256, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(256, 512),
    ToTensorV2(),
])

class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform):
        self.image_paths = sorted(glob(os.path.join(image_dir, '*.npy')))
        self.mask_paths = sorted(glob(os.path.join(mask_dir, '*.npy')))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx]).astype(np.uint8)
        mask = np.load(self.mask_paths[idx]).astype(np.uint8)
        mask[mask == 255] = 0  # 🛠️ заменяем "ignore" на фон

        augmented = self.transform(image=image, mask=mask)
        image = augmented['image'].float() / 255.0  # 🔄 нормализация в [0,1]
        mask = augmented['mask']

        return image, mask.long()

Шаг 5. Создание датасетов и загрузчиков с проверкой размеров

In [ ]:
train_dataset = SegmentationDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, train_transform)
val_dataset = SegmentationDataset(VAL_IMG_DIR, VAL_MASK_DIR, val_transform)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=0)

Train dataset size: 2975
Validation dataset size: 445


Шаг 6. Определение модели, оптимизатора, функции потерь и lr_scheduler

In [ ]:
model = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", classes=NUM_CLASSES, activation=None).to(DEVICE)
loss_fn = smp.losses.DiceLoss(mode='multiclass')
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

Шаг 7. Определение функций обучения и валидации

In [ ]:
def train_one_epoch(loader, model, optimizer, loss_fn):
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(loader):
        print(images.dtype, images.shape)
        print(masks.dtype, masks.shape)
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)


def evaluate(loader, model, loss_fn):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE).long()  # и тут тоже
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()
    return val_loss / len(loader)

Шаг 8. Запуск обучения

In [ ]:
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(train_loader, model, optimizer, loss_fn)
    val_loss = evaluate(val_loader, model, loss_fn)
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# После окончания обучения сохраняем модель
torch.save(model.state_dict(), "esnet_segmentation.pth")

  0%|          | 0/744 [00:00<?, ?it/s]

torch.float32 torch.Size([4, 3, 256, 512])
torch.int64 torch.Size([4, 256, 512])


  0%|          | 1/744 [00:15<3:15:01, 15.75s/it]

torch.float32 torch.Size([4, 3, 256, 512])
torch.int64 torch.Size([4, 256, 512])


  0%|          | 2/744 [00:30<3:05:37, 15.01s/it]

...

torch.float32 torch.Size([4, 3, 256, 512])
torch.int64 torch.Size([4, 256, 512])


  3%|▎         | 25/744 [06:03<2:55:18, 14.63s/it]

torch.float32 torch.Size([4, 3, 256, 512])
torch.int64 torch.Size([4, 256, 512])


  3%|▎         | 25/744 [06:18<3:01:13, 15.12s/it]


KeyboardInterrupt: 

# Тут будет слишком долгое обучение, облегчим параметры

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 2
NUM_CLASSES = 19
LR = 1e-3
EPOCHS = 3

2. Аугментации (с уменьшенным разрешением)

In [ ]:
train_transform = A.Compose([
    A.Resize(128, 256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(128, 256),
    ToTensorV2(),
])

3. Dataset с ограничением на 300 файлов

In [ ]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform):
        self.image_paths = sorted(glob(os.path.join(image_dir, '*.npy')))[:300]
        self.mask_paths = sorted(glob(os.path.join(mask_dir, '*.npy')))[:300]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx]).astype(np.uint8)
        mask = np.load(self.mask_paths[idx]).astype(np.uint8)
        mask = np.clip(mask, 0, NUM_CLASSES - 1)  # важно
        augmented = self.transform(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        return image.float(), mask.long()

In [ ]:
train_dataset = SegmentationDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, train_transform)
val_dataset = SegmentationDataset(VAL_IMG_DIR, VAL_MASK_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=2)

In [ ]:
model = smp.FPN(encoder_name="resnet18", classes=NUM_CLASSES, activation=None).to(DEVICE)
loss_fn = smp.losses.DiceLoss(mode='multiclass')
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
def train_one_epoch(loader, model, optimizer, loss_fn):
    model.train()
    epoch_loss = 0
    for i, (images, masks) in enumerate(tqdm(loader)):
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if i % 50 == 0:
            print(f"[Batch {i}/{len(loader)}] Loss: {loss.item():.4f}")
    return epoch_loss / len(loader)

def evaluate(loader, model, loss_fn):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()
    return val_loss / len(loader)


In [ ]:
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(train_loader, model, optimizer, loss_fn)
    val_loss = evaluate(val_loader, model, loss_fn)
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

torch.save(model.state_dict(), "segmentation_model.pth")

  1%|          | 1/150 [00:03<09:17,  3.74s/it]

[Batch 0/150] Loss: 0.7161


 34%|███▍      | 51/150 [06:35<01:55,  1.17s/it]

[Batch 50/150] Loss: 0.5330


 67%|██████▋   | 101/150 [07:23<00:46,  1.06it/s]

...

[Batch 50/150] Loss: 0.5046


 67%|██████▋   | 101/150 [01:36<00:45,  1.08it/s]

[Batch 100/150] Loss: 0.5740


100%|██████████| 150/150 [02:21<00:00,  1.06it/s]


Epoch 3/3 | Train Loss: 0.5638 | Val Loss: 0.5613
